### Evotor Data Challenge<br>Kolos Maria

In [1]:
import pandas as pd
import numpy as np

Считываем данные

In [28]:
train_df = pd.read_csv('data/evo_train.csv')
train_df.head()

,NAME,GROUP_ID,id
0,"Пиво ""Жигулевское"" 0,5 л. св.",35,0
1,СОУС ТОМАТНЫЙ БУЗДЯК 670Г ТАТАРСКИЙ /8,6,1
2,Сигареты Esse SS Exchange,7,2
3,Петрушка,6,3
4,пиво ягерь,35,4


In [29]:
test_df = pd.read_csv('data/evo_test.csv')
test_df.head()

,NAME,id
0,"Пиво ""Рижское"" светлое 0,5л",0
1,плавленый сыр король сыров 80г,1
2,заказ имл 7500342101814,2
3,Мерриес трусики М58,3
4,лупа канц мал,4


In [30]:
pd.options.mode.chained_assignment = None
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
from gensim.models.doc2vec import Doc2Vec
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
from stop_words import get_stop_words

stop_words=get_stop_words('ru')
symbols = [i for i in "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"]
sw=stop_words+symbols+['ооо','кг','мг','мл','гр','мм','см','дм','шт']
sw=set(sw)

In [32]:
import re
pattern=r'\b\d+\b'
only_letters='[^\W\d_]*'
to_be_removed = ".,:!_=><&«»?()//\\;\"\*-"
def clear_text(line):
    for c in to_be_removed:
        line = line.replace(c, ' ')
    #line=re.sub(r"(\w)([А-Я])", r"\1 \2", line)
    raw_words=re.findall(only_letters, line)#r"[\w']+"
    words=[]
    '''for word in raw_words:
        w=''.join(c for c in word if c.isalpha())
        words.append(w)'''
    '''raw_numbers=re.findall(pattern, line)
    numbers=[]
    for num in raw_numbers:
        numbers+=text_number(num)'''
    res = ' '.join(i for i in raw_words)#+' '.join(j for j in numbers)
    return res

def tokenize(text):
    #buf = ''.join(ch for ch in text)# if# ch not in exclude)
    text= clear_text(text)
    tokens = text.lower().split()   
        
    return tokens
def postprocess(data, n=1000000):
    data['tokens'] = data['NAME'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data


### Предобработка

В ходе экспериментов со стеммингом и регулярными выражениями обработка текста была сведена к минимуму (нижний регистр, очистка от пунктуации, сплит по пробелам), так как именно этот результат давал большую точность

In [33]:
train_df = postprocess(train_df)
train_df.tail()

progress-bar: 100%|███████████████████████████████████████████████████████████| 55860/55860 [00:01<00:00, 40912.66it/s]


,NAME,GROUP_ID,id,tokens
55855,"Водка ""ЦАРСКАЯ ОРИГИНАЛЬНАЯ"" 0,5",34,55855,"[водка, царская, оригинальная]"
55856,Сигареты More BY LD BLUE 25_,7,55856,"[сигареты, more, by, ld, blue]"
55857,Q10 Крем-экстра питание д/лица д/сухой и очен...,14,55857,"[q, крем, экстра, питание, д, лица, д, сухой, ..."
55858,winston XStyle,7,55858,"[winston, xstyle]"
55859,Игрушка Hasbro Star Wars фигурка Кайло Рена,26,55859,"[игрушка, hasbro, star, wars, фигурка, кайло, ..."


In [34]:
test_df = postprocess(test_df)
test_df.tail()

progress-bar: 100%|███████████████████████████████████████████████████████████| 37240/37240 [00:00<00:00, 45718.57it/s]


,NAME,id,tokens
37235,Горшок торфяной 50 *50 12 ячеек,37235,"[горшок, торфяной, ячеек]"
37236,Бюстгальтер 66071 Aveline р90D жасмин,37236,"[бюстгальтер, aveline, р, d, жасмин]"
37237,52-1067 Ложка чайная 'Юрий',37237,"[ложка, чайная, юрий]"
37238,Масло сливочное традиционное 82. 5% Из Деревеньки,37238,"[масло, сливочное, традиционное, из, деревеньки]"
37239,КОНФЕТА CHOCO FRUITS СОК ВИШНИ И ШОКОЛАД / ЖИВ...,37239,"[конфета, choco, fruits, сок, вишни, и, шокола..."


Объединяем токены в предложения для последующего подсчета tf idf матрицы

In [36]:
tfidf_corpus=[]
vocab=list(train_df.tokens.values)+list(test_df.tokens.values)
for line in vocab:
    phrase=' '.join(x for x in line)
    tfidf_corpus.append(phrase)

In [37]:
vectorizer = TfidfVectorizer()#максимальное количество слов, только униграммы
matrix = vectorizer.fit_transform(tfidf_corpus)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

vocab size : 48087


In [38]:
#обучающая и тестовая матрицы с признаками tf-idf
mxtr=matrix[:len(train_df)]
mxts=matrix[len(train_df):]

In [39]:
df=pd.DataFrame()
df['y']=train_df.GROUP_ID.values
X=matrix[list(df.index)]

In [40]:
df.head()

,y
0,35
1,6
2,7
3,6
4,35


### Стекинг

Далее проведем стекинг базовых моделей: LinearSVC, RidgeClassifier и MultinominalNB

In [41]:
def stack(clf,df,feat_name,k_folds=3):
    folds=np.split(df,k_folds)
    for i in range(k_folds):
        print(i)
        test=folds[i]
        fold_ind=[i for i in test.index]
    
        x_ind=np.array([q in fold_ind for q in df.index])
        x_test=matrix[x_ind]
        x_train=matrix[~x_ind]
        train=pd.concat([folds[k] for k in range(k_folds) if k!=i])
        clf.fit(x_train,train.y)
        
        df.loc[fold_ind,feat_name]=clf.predict(x_test)
    return df

Методы SVM хороши тем, что решают задачу, имеющую единственное решение, что обеспечивает его оптимальность. Кроме того, линейный SVC хорошо работает на тектовых данных из-за высокой размерности 

In [42]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

svc=LinearSVC(C=1)
svc.fit(mxtr,df.y)
pred_svc=svc.predict(mxts)

In [43]:
df=stack(svc,df,'svc')

0
1
2


Бинаризуем слова для дальнейшего использования метода наивного Байеса

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer()

vmatrix=cv.fit_transform(tfidf_corpus)

In [45]:
#тестовая и обучающая матрицы
vmxtr=vmatrix[:len(df)]
vmxts=vmatrix[len(df):]

Наивный байесовский классификатор оперирует с вероятностями признаков. Ключевое значение здесь играет использование метода максимального правдоподобия, который эффективно работает на текстовых данных.

In [47]:
from sklearn.naive_bayes import MultinomialNB

mnb=MultinomialNB(alpha=0.11)
mnb.fit(vmxtr,df.y)
pred_mnb=mnb.predict(vmxts)

In [48]:
df=stack(mnb,df,'mnb')

0
1
2


RidgeClassifier - еще один линейный метод, использущий l2-регуляризацию. Хорошо работает на данных с высокой размерностью

In [49]:
from sklearn.linear_model import RidgeClassifier


rc=RidgeClassifier(alpha=0.67)
rc.fit(mxtr,df.y)
pred_rc=rc.predict(mxts)
#print(accuracy_score(test_y,pred_rc))

In [50]:
df=stack(rc,df,'rc')

0
1
2


In [51]:
df.head()

,y,svc,mnb,rc
0,35,35.0,35.0,35.0
1,6,6.0,6.0,6.0
2,7,7.0,7.0,7.0
3,6,30.0,30.0,30.0
4,35,35.0,35.0,35.0


In [52]:
df_test_=pd.DataFrame()
#df_test_['y']=test_y
df_test_['svc']=list(pred_svc)
df_test_['mnb']=list(pred_mnb)
df_test_['rc']=list(pred_rc)
df_test_.head()

,svc,mnb,rc
0,35,35,35
1,6,6,6
2,9,9,9
3,26,26,26
4,24,24,24


Объединение базовых моделей 

In [53]:
from scipy.sparse import hstack

xtrain=hstack([mxtr,df[['svc','mnb','rc']].values])
xtest=hstack([mxts,df_test_[['svc','mnb','rc']].values])

### Агрегация

В качестве модели второго уровня используется xgboost. Градиентный бустинг на деревьях был выбран в качестве аггрегирующего метода поверх базовых классификаторов за счет его способности минимизации ошибок и устойчивости к переобучению. xgboost - самая эффективная и высокопроизводительная реализация градиентого бустинга.

In [54]:
from xgboost import XGBClassifier

xgb=XGBClassifier(gamma=0.15,n_estimators=400)
xgb.fit(xtrain, df.y)
predicted=xgb.predict(xtest)

Сохранение решения в файл

In [57]:
#id,GROUP_ID
text_file = open("evooutput2.csv", "w")
text_file.write("id,GROUP_ID\n")
for i in range(len(predicted)):
    text_file.write(str(test_df.loc[i].id)+','+str(predicted[i])+'\n')
text_file.close()